# New York Hotels Analysis

Allison and Amy are poor students, and they are taking a 5-day trip to New York City during spring break. They hope to find the hotel with acceptable price and convenient locations so that they won't waste too much time in the crazy NY subway. 

In [1]:
import pandas as pd
!pip install folium
import folium
import numpy as np

In [2]:
hotels = pd.read_excel('/Users/allisonxu/Desktop/NY/NYhotels.xlsx')
hotels = hotels.sort_values('Price')
hotels

,Name,Chinese Name,Price,Year,Area,Bed,Location
2,"Club Quarters Hotel, World Trade Center",世界贸易中心俱乐部住宅酒店,5732,2012,18,1*1.51,"40.710042121219715, -74.01341868855131"
5,Hyatt Grand Central New York,纽约中央Hyatt大酒店,6061,2011,26,1*1.8,"40.75244772834378, -73.97535536581093"
0,VOCO Times Square South New York,纽约时代广场南voco,6082,2021,20,1*1.8,"40.75460398889576, -73.99336281368677"
1,New York Marriott Downtown,纽约市中心万豪酒店,6112,2019,29,2*1.31,"40.70971575678169, -74.014250168961"
3,The Washington Hotel NYC,纽约华盛顿酒店,6141,2022,31.9,1*1.9,"40.70952665733615, -74.01325644807153"
4,AC Hotel New York Downtown,纽约市中心万豪AC酒店,6266,2018,22-40,1*1.8,"40.70587160203378, -74.00472977875626"


### Task 1: Mapping

In [3]:
hotels = hotels.assign(Latitude = hotels.get('Location').apply(lambda x: x.split(',')[0].strip()))
hotels = hotels.assign(Longitude = hotels.get('Location').apply(lambda x: x.split(',')[1].strip()))
hotels

,Name,Chinese Name,Price,Year,Area,Bed,Location,Latitude,Longitude
2,"Club Quarters Hotel, World Trade Center",世界贸易中心俱乐部住宅酒店,5732,2012,18,1*1.51,"40.710042121219715, -74.01341868855131",40.710042121219715,-74.01341868855131
5,Hyatt Grand Central New York,纽约中央Hyatt大酒店,6061,2011,26,1*1.8,"40.75244772834378, -73.97535536581093",40.75244772834378,-73.97535536581093
0,VOCO Times Square South New York,纽约时代广场南voco,6082,2021,20,1*1.8,"40.75460398889576, -73.99336281368677",40.75460398889576,-73.99336281368677
1,New York Marriott Downtown,纽约市中心万豪酒店,6112,2019,29,2*1.31,"40.70971575678169, -74.014250168961",40.70971575678169,-74.014250168961
3,The Washington Hotel NYC,纽约华盛顿酒店,6141,2022,31.9,1*1.9,"40.70952665733615, -74.01325644807153",40.70952665733615,-74.01325644807153
4,AC Hotel New York Downtown,纽约市中心万豪AC酒店,6266,2018,22-40,1*1.8,"40.70587160203378, -74.00472977875626",40.70587160203378,-74.00472977875626


In [4]:
hotels_center = [sum(hotels.get('Latitude').apply(lambda x: float(x)))/hotels.shape[0], \
                sum(hotels.get('Longitude').apply(lambda x: float(x)))/hotels.shape[0]]

m = folium.Map(
    location=hotels_center,
    zoom_start=13,
)

In [5]:
for _, row in hotels.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=f"""
        <b>{row['Name']}</b><br>
        Price: {row['Price']}<br>
        Area: {row['Area']}
        """,
        tooltip=row["Name"]
    ).add_to(m)
m


To include the price factor, I marked the hotels with price higher than 6,100 as red, and others as green. 

In [6]:
for _, row in hotels.iterrows():
    
    if row["Price"] > 6100:
        color = "red"
    else:
        color = "green"
    
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"{row['Name']}<br>Price: {row['Price']}"
    ).add_to(m)

m


With a closer look, three hotels located at approximatedly the same place, namely: 
* Club Quarters Hotel, World Trade Center;
* New York Marriott Downtown;
* The Washington Hotel NYC.<br>(ranked from price low to high)

In [7]:
hotels

,Name,Chinese Name,Price,Year,Area,Bed,Location,Latitude,Longitude
2,"Club Quarters Hotel, World Trade Center",世界贸易中心俱乐部住宅酒店,5732,2012,18,1*1.51,"40.710042121219715, -74.01341868855131",40.710042121219715,-74.01341868855131
5,Hyatt Grand Central New York,纽约中央Hyatt大酒店,6061,2011,26,1*1.8,"40.75244772834378, -73.97535536581093",40.75244772834378,-73.97535536581093
0,VOCO Times Square South New York,纽约时代广场南voco,6082,2021,20,1*1.8,"40.75460398889576, -73.99336281368677",40.75460398889576,-73.99336281368677
1,New York Marriott Downtown,纽约市中心万豪酒店,6112,2019,29,2*1.31,"40.70971575678169, -74.014250168961",40.70971575678169,-74.014250168961
3,The Washington Hotel NYC,纽约华盛顿酒店,6141,2022,31.9,1*1.9,"40.70952665733615, -74.01325644807153",40.70952665733615,-74.01325644807153
4,AC Hotel New York Downtown,纽约市中心万豪AC酒店,6266,2018,22-40,1*1.8,"40.70587160203378, -74.00472977875626",40.70587160203378,-74.00472977875626


### Task 2: Price to Area Ratio

In [18]:
# Clean Area and convert to float
def clean_area(area):
    area = str(area)
    if "-" in area:
        nums = area.split("-")
        return (float(nums[0]) + float(nums[1])) / 2
    else:
        return float(area)

hotels["Area_clean"] = hotels["Area"].apply(clean_area)

# Compute price per area
hotels["Price/Area"] = hotels["Price"] / hotels["Area_clean"]
hotels

,Name,Chinese Name,Price,Year,Area,Bed,Location,Latitude,Longitude,Area_clean,Price/Area
2,"Club Quarters Hotel, World Trade Center",世界贸易中心俱乐部住宅酒店,5732.0,2012,18,1*1.51,"40.710042121219715, -74.01341868855131",40.710042121219715,-74.01341868855131,18.0,318.444444
5,Hyatt Grand Central New York,纽约中央Hyatt大酒店,6061.0,2011,26,1*1.8,"40.75244772834378, -73.97535536581093",40.75244772834378,-73.97535536581093,26.0,233.115385
0,VOCO Times Square South New York,纽约时代广场南voco,6082.0,2021,20,1*1.8,"40.75460398889576, -73.99336281368677",40.75460398889576,-73.99336281368677,20.0,304.100000
1,New York Marriott Downtown,纽约市中心万豪酒店,6112.0,2019,29,2*1.31,"40.70971575678169, -74.014250168961",40.70971575678169,-74.014250168961,29.0,210.758621
3,The Washington Hotel NYC,纽约华盛顿酒店,6141.0,2022,31.9,1*1.9,"40.70952665733615, -74.01325644807153",40.70952665733615,-74.01325644807153,31.9,192.507837
4,AC Hotel New York Downtown,纽约市中心万豪AC酒店,6266.0,2018,22-40,1*1.8,"40.70587160203378, -74.00472977875626",40.70587160203378,-74.00472977875626,31.0,202.129032


We are aiming for less price and greater area, so the smaller the ratio is, the better this option is. 

In [20]:
hotels.sort_values('Price/Area').head(3)

,Name,Chinese Name,Price,Year,Area,Bed,Location,Latitude,Longitude,Area_clean,Price/Area
3,The Washington Hotel NYC,纽约华盛顿酒店,6141.0,2022,31.9,1*1.9,"40.70952665733615, -74.01325644807153",40.70952665733615,-74.01325644807153,31.9,192.507837
4,AC Hotel New York Downtown,纽约市中心万豪AC酒店,6266.0,2018,22-40,1*1.8,"40.70587160203378, -74.00472977875626",40.70587160203378,-74.00472977875626,31.0,202.129032
1,New York Marriott Downtown,纽约市中心万豪酒店,6112.0,2019,29,2*1.31,"40.70971575678169, -74.014250168961",40.70971575678169,-74.014250168961,29.0,210.758621
